In [1]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
import numpy as np
import plotly.express as px
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from scipy.stats import norm
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.regularizers import l2

In [2]:
stock_price = pd.read_csv("../data/Singtel_stock_data.csv")

In [3]:
def stock_data_manipulation(stock_price):
    stock_price.dropna(inplace = True)
    stock_price['Date'] = pd.to_datetime(stock_price['Date'], format = '%Y-%m-%d')
    stock_price['Adj Close'] = pd.to_numeric(stock_price['Adj Close'], errors = 'coerce')
    stock_price['Log Return'] = np.log(stock_price['Adj Close'] / stock_price['Adj Close'].shift(1))
    stock_price.dropna(inplace = True)
    stock_price.set_index('Date', inplace = True)
    
    return stock_price

In [4]:
stock_price = stock_data_manipulation(stock_price)

In [5]:
fig_price = px.line(stock_price, x=stock_price.index, y='Adj Close', title='Adjusted Close Time Series Plot', labels={'Adj Close': 'Adjusted Close Price'})
fig_price.show()

In [6]:
def log_return_plot(stock_price):
    fig_log = px.line(stock_price, x=stock_price.index, y = 'Log Return', title = 'Log Return')
    fig_log.update_traces(line=dict(width = 1))
    
    return fig_log

In [7]:
fig_log = log_return_plot(stock_price)
fig_log.show()

In [8]:
def rolling_vol_plot(stock_price):
    # Calculate rolling volatility (standard deviation of log returns)
    window = 30  # 30-day rolling window
    stock_price['Rolling Volatility'] = stock_price['Log Return'].rolling(window=window).std()

    fig_volatility_risk = px.line(stock_price, x=stock_price.index, y='Rolling Volatility', 
                                  title='Rolling Volatility (Risk) Over Time',
                                  labels={'Date': 'Date', 'Rolling Volatility': 'Volatility'})
    return fig_volatility_risk

In [9]:
fig_volatility_risk = rolling_vol_plot(stock_price)
fig_volatility_risk.show()

In [10]:
def volatility_pred(stock_price):
    # Build GARCH model
    am = arch_model(stock_price['Log Return']*100, vol='GARCH', p=1, q=1)
    res = am.fit(disp='off')

    # Predict the volatility of next 90 days
    forecast = res.forecast(horizon=90)

    # Extract the variance of the prediction
    variance = forecast.variance.values[-1,:]

    # Calculation of conditional standard deviation (volatility)
    cond_vol = np.sqrt(variance)

    # Create date index
    forecast_index = pd.date_range(start=stock_price.index[-1], periods=90, freq='D')

    # Assuming forecast_index and cond_vol are defined as in the previous context
    fig_volatility_pred = px.line(x=forecast_index, y=cond_vol, labels={'x': 'Date', 'y': 'Conditional volatility'}, 
                                  title='The next 90 day volatility predicted by the GARCH model')
    return fig_volatility_pred

In [11]:
fig_volatility_pred = volatility_pred(stock_price)
fig_volatility_pred.show()

In [12]:
# Prepare the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = stock_price['Adj Close'].values.reshape(-1, 1)
scaled_data = scaler.fit_transform(data)

# Create a function to prepare the dataset for LSTM
def create_dataset(data, time_step=60):
    X, Y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i - time_step:i, 0])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)

# Prepare training data
time_step = 60
X, Y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data into training and validation sets
train_size = int(len(X) * 0.8)
X_train, X_valid = X[:train_size], X[train_size:]
Y_train, Y_valid = Y[:train_size], Y[train_size:]

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, Y_train, batch_size=1, epochs=5)

# Validate the model
predictions = model.predict(X_valid)

# Reverse scaling to get the original values
predictions = scaler.inverse_transform(predictions)
Y_valid = scaler.inverse_transform(Y_valid.reshape(-1, 1))

# Plot the predictions using plotly.express
validation_df = pd.DataFrame({
    'Date': stock_price.index[train_size + time_step:],
    'Actual Price': Y_valid.flatten(),
    'Predicted Price': predictions.flatten()
})


c:\Users\LJM\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/5
927/927 ━━━━━━━━━━━━━━━━━━━━ 33s 32ms/step - loss: 0.0040
Epoch 2/5
927/927 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - loss: 8.9174e-04
Epoch 3/5
927/927 ━━━━━━━━━━━━━━━━━━━━ 33s 36ms/step - loss: 6.7739e-04
Epoch 4/5
927/927 ━━━━━━━━━━━━━━━━━━━━ 47s 50ms/step - loss: 6.0498e-04
Epoch 5/5
927/927 ━━━━━━━━━━━━━━━━━━━━ 50s 53ms/step - loss: 4.6254e-04
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step


In [13]:
fig_lstm = px.line(validation_df, x='Date', y=['Actual Price', 'Predicted Price'], title='Actual vs Predicted Stock Prices')
fig_lstm.update_layout(xaxis_title='Date', yaxis_title='Close Price')
fig_lstm.show()

In [15]:
# 准备数据
scaler = MinMaxScaler(feature_range=(0, 1))
data = stock_price['Adj Close'].values.reshape(-1, 1)
scaled_data = scaler.fit_transform(data)

# 创建函数来准备LSTM的数据集
def create_dataset(data, time_step=60):
    X, Y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i - time_step:i, 0])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)

# 准备训练数据
time_step = 60
X, Y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

# 将数据分为训练集和验证集
train_size = int(len(X) * 0.8)
X_train, X_valid = X[:train_size], X[train_size:]
Y_train, Y_valid = Y[:train_size], Y[train_size:]

# 构建LSTM模型，加入防止过拟合的层
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # 添加Dropout层，丢弃20%的神经元
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))  # 添加Dropout层
model.add(Dense(25))
model.add(Dense(1))
model.add(BatchNormalization())
# model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1), kernel_regularizer=l2(0.001)))

# 定义优化器，降低学习率
optimizer = Adam(learning_rate=0.001)

# 编译模型
model.compile(optimizer=optimizer, loss='mean_squared_error')

# 设置早停法
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# 训练模型，加入验证集和早停回调
model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), 
          epochs=100, batch_size=32, callbacks=[early_stop], verbose=1)

# 验证模型
predictions = model.predict(X_valid)

# 反归一化，得到原始数值
predictions = scaler.inverse_transform(predictions)
Y_valid = scaler.inverse_transform(Y_valid.reshape(-1, 1))

# 使用plotly.express绘制预测结果
validation_df = pd.DataFrame({
    'Date': stock_price.index[train_size + time_step:],
    'Actual Price': Y_valid.flatten(),
    'Predicted Price': predictions.flatten()
})

c:\Users\LJM\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 286ms/step - loss: 0.2519 - val_loss: 0.3321
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - loss: 0.0941 - val_loss: 0.2741
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step - loss: 0.0740 - val_loss: 0.2350
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 173ms/step - loss: 0.0594 - val_loss: 0.2157
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 152ms/step - loss: 0.0459 - val_loss: 0.1852
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - loss: 0.0367 - val_loss: 0.1860
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - loss: 0.0291 - val_loss: 0.1777
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - loss: 0.0223 - val_loss: 0.1586
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - loss: 0.0179 - val_loss: 0.1513
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - loss: 0.0127 - val_loss: 0.1209
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - loss: 0.0109 - val_loss: 0.0912
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 

In [16]:
fig_lstm = px.line(validation_df, x='Date', y=['Actual Price', 'Predicted Price'], title='Actual vs Predicted Stock Prices')
fig_lstm.update_layout(xaxis_title='Date', yaxis_title='Close Price')
fig_lstm.show()

In [17]:
# Prepare the data for prediction for the next 90 days
last_60_days = scaled_data[-time_step:]
future_predictions = []

for _ in range(90):
    input_data = last_60_days.reshape(1, time_step, 1)
    predicted_value = model.predict(input_data)
    future_predictions.append(predicted_value[0, 0])
    last_60_days = np.append(last_60_days[1:], predicted_value[0, 0])
    last_60_days = last_60_days.reshape(-1, 1)

# Inverse transform predictions
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Plot future predictions using plotly.express
future_df = pd.DataFrame({
    'Day': range(1, 91),
    'Predicted Price': future_predictions.flatten()
})

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 605ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1

In [18]:
fig_pred = px.line(future_df, x='Day', y='Predicted Price', title='Next 90 Days Stock Price Forecast')
fig_pred.update_layout(xaxis_title='Days', yaxis_title='Close Price')
fig_pred.show()

In [19]:
# Create future dates starting from the last date in the dataset
last_date = pd.to_datetime(stock_price.index[-1])
future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 91)]

# Combine historical and future predictions
future_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted Price': future_predictions.flatten()
})
combined_df = pd.concat([validation_df[['Date', 'Actual Price', 'Predicted Price']], future_df], ignore_index=True)

# Plot combined predictions using plotly.express
fig = px.line(combined_df, x='Date', y=['Actual Price', 'Predicted Price'], title='Actual and Predicted Stock Prices with Future Forecast')
fig.update_layout(xaxis_title='Date', yaxis_title='Close Price')
fig.show()

In [ ]:
# Calculate daily yield
stock_price['Return'] = stock_price['Adj Close'].pct_change().dropna()

# Calculate VaR and CVaR
confidence_level = 0.95
z_alpha = norm.ppf(1 - confidence_level)

# Calculate the mean and standard deviation of historical returns
mean_return = stock_price['Return'].mean()
std_return = stock_price['Return'].std()

# Calculate VaR（95% CI）
VaR_95 = mean_return + z_alpha * std_return

# Calculate CVaR（95% CI）
CVaR_95 = mean_return - (norm.pdf(z_alpha) / (1 - confidence_level))

# Use GARCH model to predict future volatility
model = arch_model(stock_price['Return'].dropna(), vol='Garch', p=1, q=1)
results = model.fit()

# Predict volatility
future_volatility = results.conditional_volatility[-1] 

# Calculate VaR and CVaR based on predicted volatility
VaR_95_forecast = mean_return + z_alpha * future_volatility
CVaR_95_forecast = mean_return - (norm.pdf(z_alpha) / (1 - confidence_level)) * future_volatility

# Visualizationg
fig_risk = px.line(stock_price, x=stock_price.index, y='Return', title='VaR and CVaR Calculation', labels={'Return': 'Returns'})
fig_risk.add_hline(y=VaR_95, line_dash="dash", line_color="red", annotation_text=f'VaR (95%): {VaR_95:.4f}', annotation_position="top right")
# fig_risk.add_hline(y=CVaR_95, line_dash="dash", line_color="blue", annotation_text=f'CVaR (95%): {CVaR_95:.4f}', annotation_position="top right")
fig_risk.add_hline(y=VaR_95_forecast, line_dash="dash", line_color="green", annotation_text=f'Future VaR (95%): {VaR_95_forecast:.4f}', annotation_position="top right")
fig_risk.add_hline(y=CVaR_95_forecast, line_dash="dash", line_color="purple", annotation_text=f'Future CVaR (95%): {CVaR_95_forecast:.4f}', annotation_position="top right")

fig_risk.show()

c:\Users\LJM\AppData\Local\Programs\Python\Python312\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001567. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


C:\Users\LJM\AppData\Local\Temp\ipykernel_30976\390438338.py:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Iteration:      1,   Func. Count:      5,   Neg. LLF: -3669.1887001350474
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3669.188704311228
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 1
